# PDIoT Group A1 model

### Imports

In [1]:
import os
import numpy as np
import tensorflow as tf

from ml.config import cfg
from ml.dataset import odgt2data
from ml.models import ModelBuilder, OptimizerBuilder, LRScheduleBuilder

print("All packages imported!")

2023-11-10 00:57:44.373686: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


All packages imported!


### Config

In [2]:
config_fp = '/home/jonathan/git/pdiot-ml/cfg/mlp-motion.yaml'
cfg.merge_from_file(config_fp)
cfg.DATASET.path = '/home/jonathan/git/pdiot-ml/data/sets/pdiot-data/'
cfg.TRAIN.path = '/home/jonathan/git/pdiot-ml/ckpt/mlp-motion/'   
print(cfg)

DATASET:
  LIST:
    test: test_motion_pdiot-data.odgt
    train: train_motion_pdiot-data.odgt
    val: val_motion_pdiot-data.odgt
  num_classes: 2
  path: /home/jonathan/git/pdiot-ml/data/sets/pdiot-data/
MODEL:
  ARCH:
    LSTM:
      hidden_size: 16
      num_layers: 0
    MLP:
      dropout: 0.2
      hidden_size: 32
      num_layers: 3
  INPUT:
    format: summary
    sensor: all
    window_size: 15
TRAIN:
  DATA:
    batch_size: 128
    overlap_size: 5
  FN:
    config: config.yaml
    history: history.csv
    log: log.txt
    weight: weights.hdf5
  LEN:
    early_stop: 10
    num_epoch: 100
  LR:
    gamma: 0.1
    schedule: step
    step_size: 40
  OPTIM:
    lr: 0.001
    momentum: 0.9
    optim: adam
    weight_decay: 0.0005
  path: /home/jonathan/git/pdiot-ml/ckpt/mlp-motion/


### Data

In [3]:
train_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.train)
val_odgt_fp = os.path.join(cfg.DATASET.path, cfg.DATASET.LIST.val)

train_X, train_y = odgt2data(train_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                            cfg.TRAIN.DATA.overlap_size, 
                                            cfg.DATASET.num_classes)
val_X, val_y = odgt2data(val_odgt_fp, cfg.MODEL.INPUT.window_size, 
                                      cfg.TRAIN.DATA.overlap_size,
                                      cfg.DATASET.num_classes)

### Model

In [4]:
model = ModelBuilder.build_classifier(cfg.MODEL, '', cfg.DATASET.num_classes)
optimizer = OptimizerBuilder.build_optimizer(cfg.TRAIN.OPTIM)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = [
    tf.keras.metrics.SparseCategoricalAccuracy(name='acc'),
]

lr_scheduler = LRScheduleBuilder.build_scheduler(cfg.TRAIN.LR)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

early_stop_callback = tf.keras.callbacks.EarlyStopping(monitor='val_acc', patience=cfg.TRAIN.LEN.early_stop)

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(cfg.TRAIN.path, 'weights.hdf5'),
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)

history_callback = tf.keras.callbacks.CSVLogger(
    os.path.join(cfg.TRAIN.path, 'history.csv'), 
    separator=',', 
    append=False)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
model.fit(train_X, train_y, validation_data=(val_X, val_y), 
          epochs=cfg.TRAIN.LEN.num_epoch, 
          batch_size=cfg.TRAIN.DATA.batch_size,
          callbacks=[lr_callback, 
                     early_stop_callback,
                     checkpoint_callback,
                     history_callback])

2023-11-10 00:57:49.960404: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-10 00:57:49.961796: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/100
1101/1101 [==============================] - 8s 7ms/step - loss: 0.2432 - acc: 0.9120 - val_loss: 0.1097 - val_acc: 0.9597 - lr: 0.0010
Epoch 2/100
1101/1101 [==============================] - 9s 8ms/step - loss: 0.0922 - acc: 0.9646 - val_loss: 0.0814 - val_acc: 0.9679 - lr: 0.0010
Epoch 3/100
1101/1101 [==============================] - 8s 8ms/step - loss: 0.0657 - acc: 0.9763 - val_loss: 0.0525 - val_acc: 0.9786 - lr: 0.0010
Epoch 4/100
1101/1101 [==============================] - 10s 9ms/step - loss: 0.0568 - acc: 0.9794 - val_loss: 0.0525 - val_acc: 0.9765 - lr: 0.0010
Epoch 5/100
1101/1101 [==============================] - 8s 8ms/step - loss: 0.0543 - acc: 0.9807 - val_loss: 0.0503 - val_acc: 0.9805 - lr: 0.0010
Epoch 6/100
1101/1101 [==============================] - 9s 8ms/step - loss: 0.0519 - acc: 0.9814 - val_loss: 0.0461 - val_acc: 0.9808 - lr: 0.0010
Epoch 7/100
1101/1101 [==============================] - 9s 8ms/step - loss: 0.0502 - acc: 0.9822 - val_loss: 0

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 12)                0         
                                                                 
 dense (Dense)               (None, 32)                416       
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                1056      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0